In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os

In [2]:
working_dir = os.getcwd()

In [3]:
def get_one_contract_data_by_date(date_str, contract_pd):
    """Get the contract minute data by the date
    Trading hours:
        Sunday – Friday, 7:00 p.m. – 7:45 a.m. CT and 
        Monday – Friday, 8:30 a.m. – 1:20 p.m. CT
    Define the last day's night-time and today's day-time as today's trading hours
    
    Args:
        date_str: str, the date we want to extract the main contract data, e.g. "2016-01-04"
        contract_path: str, the main contract path, e.g. "../data/ZC/ZCH16.csv"
        
    Returns:
        pandas.DataFrame
    
    """
    year, month, day = map(int, date_str.split('-'))
    last_date_str = (datetime(year, month, day) - timedelta(days=1)).strftime("%Y-%m-%d")
    
    # all_data = pd.read_csv(contract_path, header=0, index_col=0, parse_dates=[0])
    all_data = contract_pd
    all_data['Volume'] = all_data.TotalVolume.diff()
    
    last_day_data = all_data.loc[last_date_str].between_time("19:00", "23:59")
    today_data = all_data.loc[date_str].between_time("0:00", "13:20")
    
    data = last_day_data.append(today_data, ignore_index=False)
    data.iloc[0, 5] = data.iloc[0, 4] #The first minute's trading volume 
    return data


In [4]:
def extract_data_from_unique_contract(contract_path, unique_dates):
    contract_pd = pd.read_csv(contract_path, header=0, index_col=0, parse_dates=[0])
    data = pd.DataFrame(columns=["Open", "High", "Low", "Close", "TotalVolume", "Volume"])
    for single_date in unique_dates:
        dd = get_one_contract_data_by_date(single_date, contract_pd)
        data = data.append(dd, ignore_index=False)
    return data

In [5]:
def get_all_main_contract_data(main_contracts_path = working_dir+"/data/ZC/front.csv", contracts_root_path = working_dir+"/data/ZC/"):
    """Combine the daily main contract data 
    Args:
        main_contracts_path: str, the path that have the daily main contract code
        contracts_root_path: str, the root path that have all the contract data
        
    Returns:
        data, pd.DataFrame,columns=["Open", "High", "Low", "Close", "TotalVolume", "Volume", "Change"], 
                            where "Change" == 1 indicate that the main contract changed that day
    
    """
    func_contract_to_path = lambda x: contracts_root_path + "ZC" + x + ".csv"
    
    main_contracts = pd.read_csv(main_contracts_path, header=None, names=['date_str', 'main_contract'])
    unique_contracts = main_contracts.main_contract.unique()
    
    data = pd.DataFrame(columns=["Open", "High", "Low", "Close", "TotalVolume", "Volume"])
    for single_contract in unique_contracts:
        contract_path = func_contract_to_path(single_contract)
        unique_dates = main_contracts.loc[main_contracts["main_contract"] == single_contract].date_str.unique()
        today_data = extract_data_from_unique_contract(contract_path, unique_dates)
        data = data.append(today_data, ignore_index=False)
        
    return data

In [7]:
%time all_data = get_all_main_contract_data()

Wall time: 13.7 s


In [8]:
all_data.to_csv(working_dir+"/data/combined_data.csv")

In [9]:
len(all_data)

535536

In [10]:
# Get the main contracts changing date
main_contracts = pd.read_csv(working_dir+"/data/ZC/front.csv", header=None, names=['date_str', 'main_contract'])
main_contracts['Change'] = main_contracts.main_contract.apply(lambda x: hash(x)).rolling(2).apply(lambda x: 1 if x[1] != x[0] else 0).fillna(1)
change_dates = main_contracts[main_contracts['Change'] == 1][['date_str', 'main_contract']].reset_index(drop=True)
change_dates


C:\Users\renhao\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  This is separate from the ipykernel package so we can avoid doing imports until


,date_str,main_contract
0,2016-01-04,H16
1,2016-02-24,K16
2,2016-04-20,N16
3,2016-06-29,U16
4,2016-08-17,Z16
5,2016-11-29,H17
6,2017-02-27,K17
7,2017-04-20,N17
8,2017-06-28,U17
9,2017-08-17,Z17


In [12]:
change_dates.to_csv(working_dir+"/data/main_contract_change_dates.csv", index=False)